# Analisis Anomali Network Menggunakan FP-Growth dan Association Rules

## Deskripsi Proyek
Notebook ini menganalisis log jaringan untuk mendeteksi anomali menggunakan algoritma **FP-Growth** (Frequent Pattern Growth) dan **Association Rules**. Tujuan utama adalah menemukan pola hubungan antara kondisi jaringan dan diagnosis anomali.

## Struktur Analisis
1. **Encoding Data**: Mengonversi data log menjadi format transaksi yang dapat dianalisis
2. **FP-Growth Algorithm**: Menemukan pola frekuen dengan threshold minimum support
3. **Association Rules**: Menghasilkan aturan asosiasi untuk prediksi diagnosis
4. **Filtering & Visualization**: Menyaring aturan yang relevan dan menampilkan hasil akhir

## Tahap 1: Import Library dan Persiapan Data

Pada tahap ini, kami mengimpor library yang diperlukan untuk analisis frequent pattern mining dan memuat dataset yang sudah dibersihkan dari tahap preprocessing.

In [1]:
import pandas as pd
import ast
import gc
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

# Konfigurasi Global
INPUT_FILE = 'Data/Data_Siap_Mining.csv'
OUTPUT_FILE = 'Final_Skripsi_Rules.csv'
MIN_SUPPORT_COUNT = 30
MIN_CONFIDENCE = 0.6

print("Library & Konfigurasi Siap.")

Library & Konfigurasi Siap.


## Tahap 2: Encoding Data dan Perhitungan FP-Growth

### Proses Encoding
- **TransactionEncoder**: Mengkonversi data log dari format text menjadi format transaksi (one-hot encoded)
- Setiap item dalam 'items' column diubah menjadi kolom boolean terpisah
- Output: DataFrame `df_encoded` dengan nilai True/False untuk setiap atribut

### Perhitungan Support
- **Minimum Support**: Threshold minimum untuk pola yang dianggap "sering" terjadi
- **MIN_SUPPORT_COUNT = 20**: Pola harus muncul minimal 20 kali dalam dataset
- **min_support_pct**: Persentase support dihitung dari total jumlah transaksi

### Algoritma FP-Growth
- **fpgrowth**: Algoritma efisien untuk mining pola frekuen
- Lebih cepat dari Apriori karena menggunakan FP-tree structure
- Output: `frequent_itemsets` berisi semua pola yang memenuhi threshold

### Association Rules
- Menghasilkan aturan IF-THEN dari pola frekuen
- **min_threshold = 0.6**: Hanya aturan dengan confidence ≥ 60% yang dipertahankan
- Metrik yang dihitung: support, confidence, lift, dsb.

In [2]:
# CELL 2: Load Data & Gabung Label (CRITICAL STEP)
# 1. Load the dataset from INPUT_FILE
df = pd.read_csv(INPUT_FILE)
print(f" Dataset dimuat: {len(df)} baris, {df.shape[1]} kolom")

# 2. Determine if the label column is named 'Label' or 'diagnosis'
label_col = 'Label' if 'Label' in df.columns else 'diagnosis'
print(f" Kolom Label: {label_col}")

# 3. Create a list called 'transactions'
# 4. For each row, parse the 'items' string using ast.literal_eval (NOT eval)
# 5. CRITICAL: Append the row's Label/diagnosis value into the items list
# 6. Add the list to 'transactions'
transactions = []
for _, row in df.iterrows():
    # Parse string list kembali ke list python menggunakan ast.literal_eval (AMAN)
    items = ast.literal_eval(row['items']) if isinstance(row['items'], str) else row['items']
    # CRITICAL: Masukkan Label ke dalam items list (INI KUNCINYA!)
    items.append(row[label_col])
    transactions.append(items)

# 7. Delete the dataframe and run gc.collect() to free up memory
del df
gc.collect()

# 8. Print the first transaction to verify the label is included
print(f"Transaksi siap. Contoh baris pertama: {transactions[0]}")
print(f"Total transaksi: {len(transactions)}")

 Dataset dimuat: 6537 baris, 2 kolom
 Kolom Label: Label
Transaksi siap. Contoh baris pertama: ['r1-core', 'system', 'router', 'rebooted', 'console', 'admin', 'ttys0', 'shutdown', 'NORMAL']
Total transaksi: 6537


In [3]:
# CELL 3: Encoding & FP-Growth
# 1. Initialize TransactionEncoder. Fit and transform 'transactions' into a boolean array
print("Sedang melakukan Encoding...")
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)

# 2. Delete 'transactions' list and run gc.collect() immediately to save RAM
del transactions
gc.collect()

# 3. Create a DataFrame 'df_encoded' from the array. Delete the array and run gc.collect()
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)
del te_ary
gc.collect()

# 4. Calculate 'min_support_pct' by dividing MIN_SUPPORT_COUNT by the length of df_encoded
min_support_pct = MIN_SUPPORT_COUNT / len(df_encoded)
print(f"Encoding Selesai. Shape: {df_encoded.shape}")
print(f"Mining dengan Min Support: {min_support_pct:.5f} (Min {MIN_SUPPORT_COUNT} kejadian)")

# 5. Run fpgrowth on 'df_encoded' using 'min_support_pct' and use_colnames=True
frequent_itemsets = fpgrowth(df_encoded, min_support=min_support_pct, use_colnames=True)

# 6. Delete 'df_encoded' and run gc.collect()
del df_encoded
gc.collect()

# 7. Print the number of frequent itemsets found
print(f"Mining Selesai. Ditemukan {len(frequent_itemsets)} pola itemset.")
print(f"Cek Jumlah Pola: {len(frequent_itemsets)}")
if len(frequent_itemsets) > 500000:
    print("Terlalu banyak pola (>500rb).")

Sedang melakukan Encoding...
Encoding Selesai. Shape: (6537, 132)
Mining dengan Min Support: 0.00459 (Min 30 kejadian)
Mining Selesai. Ditemukan 630334 pola itemset.
Cek Jumlah Pola: 630334
Terlalu banyak pola (>500rb).


## Tahap 3: Penyaringan dan Tampilan Hasil Akhir

### Konsep Filtering
Dari semua association rules yang dihasilkan, kita hanya ingin menyimpan rules yang:
- **Consequent (Kesimpulan)**: Harus berupa diagnosis/label anomali
  - NORMAL: Kondisi jaringan normal
  - UPSTREAM_FAILURE: Kegagalan uplink
  - LINK_FAILURE: Kegagalan link
  - DDOS_ATTACK: Serangan DDoS
  - BROADCAST_STORM: Broadcast storm

Contoh Rule yang BENAR (yang kita inginkan):
- IF `ether1=down AND queue=high` THEN `LINK_FAILURE` ✓

Contoh Rule yang SALAH (yang kita buang):
- IF `LINK_FAILURE` THEN `ether1=down` ✗

### Metrik Evaluasi
- **Support**: Seberapa sering pola ini muncul dalam dataset (0-1)
- **Confidence**: Probabilitas consequent terjadi jika antecedent terjadi (0-1)
- **Lift**: Seberapa kuat hubungan antara antecedent dan consequent (>1 = hubungan positif)

In [4]:
# CELL 4: Generate Rules, Filter & Save
# 1. Generate association rules using metric="confidence" and min_threshold=MIN_CONFIDENCE
print(f"\nGenerating Rules (Min Conf: {MIN_CONFIDENCE})...")
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=MIN_CONFIDENCE)

# Delete frequent_itemsets to free memory
del frequent_itemsets
gc.collect()

# 2. Define TARGET_LABELS list
TARGET_LABELS = ['NORMAL', 'UPSTREAM_FAILURE', 'LINK_FAILURE', 'DDOS_ATTACK', 'BROADCAST_STORM']

# 3. Create a filter function to check if 'consequents' contains any of the TARGET_LABELS
def is_diagnosis_rule(consequents):
    return any(label in consequents for label in TARGET_LABELS)

# 4. Filter the 'rules' dataframe to keep only rows where consequents are diagnosis labels
if not rules.empty:
    final_rules = rules[rules['consequents'].apply(is_diagnosis_rule)].copy()
else:
    final_rules = pd.DataFrame()

# 5. Sort the final rules by 'confidence' and 'support' descending
if not final_rules.empty:
    final_rules = final_rules.sort_values(['confidence', 'support'], ascending=[False, False])

    # 6. Convert 'antecedents' and 'consequents' columns from frozenset to standard Python lists
    final_rules['antecedents'] = final_rules['antecedents'].apply(lambda x: list(x))
    final_rules['consequents'] = final_rules['consequents'].apply(lambda x: list(x))

    # 7. Print the top 10 rules for preview
    print(f"\n{'='*60}")
    print(f"TOP 10 PREVIEW (Dari total {len(final_rules)} rules)")
    print(f"{'='*60}")
    for idx, row in final_rules.head(10).iterrows():
        print(f"IF {row['antecedents']} THEN {row['consequents']} (Conf: {row['confidence']:.2f})")

    # 8. Save ALL final rules to OUTPUT_FILE (do not limit to head)
    final_rules.to_csv(OUTPUT_FILE, index=False)
    
    print(f"\n{'='*60}")
    print(f"SUKSES! Total {len(final_rules)} Rules valid disimpan.")
    print(f"File: {OUTPUT_FILE}")
    print(f"{'='*60}")

else:
    print("PERINGATAN: Tidak ada rule diagnosis yang ditemukan!")
    print("Cek apakah Label/diagnosis sudah masuk ke transaksi di CELL 2.")


Generating Rules (Min Conf: 0.6)...


KeyboardInterrupt: 

## Kesimpulan dan Interpretasi

### Hasil Analisis
Notebook ini menghasilkan association rules yang menunjukkan hubungan antara kondisi jaringan dan tipe anomali. Setiap rule memiliki:
- **Antecedents (IF)**: Kondisi atau kombinasi kondisi jaringan yang diamati
- **Consequents (THEN)**: Diagnosis/prediksi tipe anomali yang kemungkinan terjadi
- **Confidence**: Tingkat akurasi prediksi (semakin tinggi semakin baik)
- **Support**: Seberapa sering pola ini muncul dalam data (frekuensi)
- **Lift**: Seberapa kuat hubungan antara kondisi dan diagnosis

### Penggunaan Praktis
Hasil rules ini dapat digunakan untuk:
1. **Early Warning System**: Mendeteksi anomali sebelum terjadi dengan mengecek kondisi antecedent
2. **Root Cause Analysis**: Memahami faktor-faktor apa yang menyebabkan setiap tipe anomali
3. **Network Optimization**: Meningkatkan monitoring dan maintenance protokol berdasarkan insights
4. **Decision Support**: Membantu network administrator membuat keputusan yang lebih cepat dan tepat

### File Output
- **Final_Skripsi_Rules.csv**: Berisi top 10 rules dalam format CSV yang dapat digunakan untuk proses selanjutnya